# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f18a0ee1f40>
├── 'a' --> tensor([[ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675]])
└── 'x' --> <FastTreeValue 0x7f18a0ee1f10>
    └── 'c' --> tensor([[ 0.2868,  1.8916, -0.4596,  1.5075],
                        [-0.1334,  0.5806,  0.4720,  0.2513],
                        [ 0.4364, -1.4575, -0.1207, -0.5282]])

In [4]:
t.a

tensor([[ 0.7894, -1.5597, -1.1827],
        [-0.8797,  0.5197, -0.9675]])

In [5]:
%timeit t.a

76.8 ns ± 0.596 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f18a0ee1f40>
├── 'a' --> tensor([[-1.3128, -0.7818, -0.6840],
│                   [-0.2585,  1.3622, -0.7194]])
└── 'x' --> <FastTreeValue 0x7f18a0ee1f10>
    └── 'c' --> tensor([[ 0.2868,  1.8916, -0.4596,  1.5075],
                        [-0.1334,  0.5806,  0.4720,  0.2513],
                        [ 0.4364, -1.4575, -0.1207, -0.5282]])

In [7]:
%timeit t.a = new_value

74.1 ns ± 0.827 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675]]),
    x: Batch(
           c: tensor([[ 0.2868,  1.8916, -0.4596,  1.5075],
                      [-0.1334,  0.5806,  0.4720,  0.2513],
                      [ 0.4364, -1.4575, -0.1207, -0.5282]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7894, -1.5597, -1.1827],
        [-0.8797,  0.5197, -0.9675]])

In [11]:
%timeit b.a

67.2 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.9425,  0.8928,  0.4996],
               [ 1.6387,  0.3291, -0.1200]]),
    x: Batch(
           c: tensor([[ 0.2868,  1.8916, -0.4596,  1.5075],
                      [-0.1334,  0.5806,  0.4720,  0.2513],
                      [ 0.4364, -1.4575, -0.1207, -0.5282]]),
       ),
)

In [13]:
%timeit b.a = new_value

624 ns ± 7.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

999 ns ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.8 µs ± 84.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

298 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

278 µs ± 7.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f17ea225b20>
├── 'a' --> tensor([[[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]],
│           
│                   [[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]],
│           
│                   [[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]],
│           
│                   [[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]],
│           
│                   [[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]],
│           
│                   [[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]],
│           
│                   [[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]],
│           
│                   [[ 0.7894, -1.5597, -1.1827],
│                    [-0.8797,  0.5197, -0.9675]]])
└── 'x' --> <FastTreeValue 0x7f17ea225df0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.1 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f17ea7ed6d0>
├── 'a' --> tensor([[ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675],
│                   [ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675],
│                   [ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675],
│                   [ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675],
│                   [ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675],
│                   [ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675],
│                   [ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675],
│                   [ 0.7894, -1.5597, -1.1827],
│                   [-0.8797,  0.5197, -0.9675]])
└── 'x' --> <FastTreeValue 0x7f17ea7ed370>
    └── 'c' --> tensor([[ 0.2868,  1.8916, -0.4596,  1.5075],
                        [-0.1334,  0.5806,  0.4720,  0.2513],
                 

In [23]:
%timeit t_cat(trees)

44.1 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.5 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]],
       
               [[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]],
       
               [[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]],
       
               [[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]],
       
               [[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]],
       
               [[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]],
       
               [[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]],
       
               [[ 0.7894, -1.5597, -1.1827],
                [-0.8797,  0.5197, -0.9675]]]),
    x: Batch(
           c: tensor([[[ 0.2868,  1.8916, -0.4596,  1.5075],
                       [-0.1334,  0.5806,  0.4720,  0.2513],
                       [ 0.4364, -1.4575, -0.1207, -0.5282]],
         

In [26]:
%timeit Batch.stack(batches)

103 µs ± 1.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675],
               [ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675],
               [ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675],
               [ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675],
               [ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675],
               [ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675],
               [ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675],
               [ 0.7894, -1.5597, -1.1827],
               [-0.8797,  0.5197, -0.9675]]),
    x: Batch(
           c: tensor([[ 0.2868,  1.8916, -0.4596,  1.5075],
                      [-0.1334,  0.5806,  0.4720,  0.2513],
                      [ 0.4364, -1.4575, -0.1207, -0.5282],
                      [ 0.2868,  1.8916, -0.4596,  1.5075],
                      [-0.1334,  

In [28]:
%timeit Batch.cat(batches)

187 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

576 µs ± 9.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
